In [ ]:
landmark_indices = [0,5,6,7,8,9,10,11,12,13]
n_landmark = len(landmark_indices)

In [ ]:
all_lm_centroid_global_rotated_versions = dict([])

for lm_ind in landmark_indices:
    with open('/home/yuncong/csd395/all_landmark_descriptors_%d.pkl'%lm_ind, 'r') as f:
            landmark_descriptor = pickle.load(f)

    lm_centroid_global_rotated_versions = map(lambda d: d['bbox'][4:6], landmark_descriptor)
    all_lm_centroid_global_rotated_versions[lm_ind] = lm_centroid_global_rotated_versions

In [ ]:
all_lm_centroid_global = dict([(ind, rvs[4]) for ind, rvs in all_lm_centroid_global_rotated_versions.iteritems()])

In [ ]:
response_maps = dict([])
for lm_ind in landmark_indices:
    response_maps[lm_ind] = dm.load_pipeline_result('responseMapLmAllRotations%d'%lm_ind, 'npy')[4]

In [ ]:
from itertools import product

In [ ]:
image_centroid = np.array((int(dm.image_width/2), int(dm.image_height/2)))
atlas_centroid = np.array((2772, 1385))

xshifts = np.linspace(-400, 400, 100)
yshifts = np.linspace(-400, 400, 100)
angles = np.linspace(-np.pi/4, np.pi/4, 17)

In [ ]:
# sample_lms = np.random.choice(range(n_landmark), n_landmark, replace=False)
sample_lms = [0, 9]
# sample_lms = landmark_indices

In [ ]:
X = [all_lm_centroid_global[i] for i in sample_lms]
Xc = X - atlas_centroid
probs = []
for xshift, yshift, angle in product(xshifts, yshifts, angles):
#     print xshift, yshift, angle
    T = np.array([[np.cos(angle), -np.sin(angle), xshift],
                    [np.sin(angle), np.cos(angle), yshift],
                    [0, 0, 1.]])
    TXc = rigid_transform_to(Xc, T).astype(np.int)
    TX = TXc + image_centroid
    prob = np.sum(response_maps[lm][TX[i,1], TX[i,0]] if TX[i,1] < dm.image_height and TX[i,1] >= 0 \
                                                        and TX[i,0] < dm.image_width and TX[i,0] >= 0\
                  else 0
                  for i, lm in enumerate(sample_lms))
    probs.append(prob)

In [ ]:
probs_map = np.reshape(probs, (len(xshifts), len(yshifts), len(angles)))

In [ ]:
xshift_opt_ind, yshift_opt_ind, rot_opt_ind = np.unravel_index(np.argmax(probs_map), probs_map.shape)
print xshift_opt_ind, yshift_opt_ind, rot_opt_ind

xshift_opt = xshifts[xshift_opt_ind]
yshift_opt = yshifts[yshift_opt_ind]
rot_opt = angles[rot_opt_ind]

print xshift_opt, yshift_opt, rot_opt, np.rad2deg(rot_opt)

In [ ]:
plt.matshow(probs_map[:,:,rot_opt_ind]);
plt.xticks(range(0, len(xshifts), 20), xshifts[::20].astype(np.int));
plt.xlabel('x-offset');
plt.yticks(range(0, len(yshifts), 20), yshifts[::20].astype(np.int));
plt.ylabel('y-offset');
plt.colorbar();

In [ ]:
from skimage.transform import warp, AffineTransform

R = np.array([[np.cos(rot_opt), -np.sin(rot_opt)],
                [np.sin(rot_opt), np.cos(rot_opt)]])

shift = image_centroid + (xshift_opt, yshift_opt) - np.dot(R, atlas_centroid)
T = np.vstack([np.column_stack([R, shift]), [0,0,1]])

trf = AffineTransform(T)
img_trf = warp(dm.image_rgb, inverse_map=trf)

In [ ]:
canvas = np.zeros_like(dms[1].image_rgb)
canvas[:dm.image_height, :dm.image_width] = img_trf

overlayed = alpha_blending(canvas, dms[1].image_rgb, .5, .5)

display(canvas)